# TF Tutorial

In [1]:
import os
import tensorflow as tf
import numpy as np


In [2]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [3]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

In [4]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size, architecture = 'LSTM', layers = 1):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                batch_input_shape=[batch_size,None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful = True,
                recurrent_initializer = 'glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [13]:
def train_model(name, EPOCHS = 100):
    if name == 'generic':
        text = open('../../texts/essays.txt', 'rb').read().decode(encoding='utf-8')
        
    else:

        text = open('../../texts/{}.txt'.format(name), 'rb').read().decode(encoding='utf-8')
        essays = open('../../texts/essays.txt', 'rb').read().decode(encoding='utf-8')

        text = text + essays
    print(len(text))
    vocab = sorted(set(text))
    
    char2idx = {unique:idx for idx, unique in enumerate(vocab)}
    idx2char = np.array(vocab)
    text_as_int = np.array([char2idx[char] for char in text])
    seq_len = 100

    examples_per_epoch = len(text) // (seq_len + 1)
    char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
    sequences = char_dataset.batch(seq_len + 1, drop_remainder = True)
    dataset = sequences.map(split_input_target)
    BATCH_SIZE = 64
    BUFFER_SIZE = 10000

    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

    vocab_size = len(vocab)
    
    embedding_dim = 256
    rnn_units = 1024
    
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = BATCH_SIZE)
    
    model.compile(optimizer = 'adam', loss = loss)
    checkpoint_path = './final_weights/{}/'.format(name)
    try:
        model.load_weights(checkpoint_path)
        print('model loaded')
    except:
        pass
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True)
    
    history = model.fit(dataset, epochs = EPOCHS, callbacks = [cp_callback])
    
    return history

In [6]:
text = open('../../texts/marx.txt', 'rb').read().decode(encoding='utf-8')
essays = open('../../texts/essays.txt', 'rb').read().decode(encoding='utf-8')

text = text + essays

In [7]:
print(len(text))

573715


In [44]:
vocab = sorted(set(text))

In [45]:
char2idx = {unique:idx for idx, unique in enumerate(vocab)}
idx2char = np.array(vocab)

In [46]:
text_as_int = np.array([char2idx[char] for char in text])

In [47]:
seq_len = 100

examples_per_epoch = len(text) // (seq_len + 1)

In [48]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [49]:
sequences = char_dataset.batch(seq_len + 1, drop_remainder = True)


In [50]:
dataset = sequences.map(split_input_target)

In [51]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

vocab_size = len(vocab)

In [52]:
embedding_dim = 256
rnn_units = 1024

In [53]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = BATCH_SIZE)

In [54]:
model.compile(optimizer = 'adam', loss = loss)

In [55]:
len(text)

573715

In [56]:
#checkpoint_dir = './training_checkpoints'
#checkpoint_prefix = os.path.join(checkpoint_dir, 'marx_{epoch}')
#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath = checkpoint_prefix, save_weights_only = True)
checkpoint_path = './final_weights/marx/'
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True)

In [57]:
EPOCHS = 100

In [58]:
history = model.fit(dataset, epochs = EPOCHS, callbacks = [cp_callback])

Train for 88 steps
Epoch 1/100
88/88 [==============================] - 5s 58ms/step - loss: 2.9391
Epoch 2/100
63/88 [====================>.........] - ETA: 1s - loss: 2.4375

KeyboardInterrupt: 

In [25]:
generative_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)

In [26]:
generative_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [27]:
generative_model.build(tf.TensorShape([1,None]))

In [38]:
def gen_text(model, start_string):
    num_gen = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    temperature = 1.0
    model.reset_states()
    
    for i in range(num_gen):
        predictions = generative_model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    while idx2char[predicted_id] != '.':
        predictions = generative_model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    ret_str = start_string + ''.join(text_generated)
    ret_str = ret_str.replace('\n', ' ')
    ret_str = ret_str.split('.')
    ret_str = '.'.join(ret_str[1:])
    return (ret_str)

In [39]:
i = int(np.random.randint(0, len(text)-100))
start_text = text[i:int(i+100)]
generated_text = gen_text(generative_model, start_text)

In [40]:
generated_text


' These “Wages of Whiteness,” as Roediger described them to discover, expecial in the production of history’s direction, his abstraction is not a mutual misunderstanding of companishtof  of the divergent person and to the absolute monarchy and "Yak at ho wine  Within Ro  some of this tas  A  discrepancy of individuals; all  they makes as he is from his fellow, although it is manipulated by his practices and are struggle into three age groups will achieved though it mythologies from the collition of bourgeois property But in the bourgeois, as well as their own cultural signifiers to gradation of cyit is self-redamental right to the proletariat are made up of time Both are es helped on diamonds and  durkhe marooon” as a multiplicity and lies him enarche’s claims, with local corresponding of society more immediately begin Spelke and Thoe chayit is not require the same abort from the first monkey recognition to fight naked by protection of its cultural outcoinsion of capital and loces that

In [40]:
history = train_model('lenin')

473662
Train for 73 steps
Epoch 1/100
73/73 [==============================] - 4s 61ms/step - loss: 2.9812
Epoch 2/100
73/73 [==============================] - 3s 44ms/step - loss: 2.4301
Epoch 3/100
73/73 [==============================] - 3s 44ms/step - loss: 2.2037
Epoch 4/100
73/73 [==============================] - 3s 45ms/step - loss: 2.0246
Epoch 5/100
73/73 [==============================] - 3s 44ms/step - loss: 1.8699
Epoch 6/100
73/73 [==============================] - 3s 47ms/step - loss: 1.7323
Epoch 7/100
73/73 [==============================] - 3s 44ms/step - loss: 1.6176
Epoch 8/100
73/73 [==============================] - 3s 45ms/step - loss: 1.5277
Epoch 9/100
73/73 [==============================] - 3s 45ms/step - loss: 1.4553
Epoch 10/100
73/73 [==============================] - 3s 44ms/step - loss: 1.3983
Epoch 11/100
73/73 [==============================] - 3s 46ms/step - loss: 1.3521
Epoch 12/100
73/73 [==============================] - 3s 46ms/step - loss: 1.3121

In [62]:
i = int(np.random.randint(0, len(text)-100))
start_text = text[i:int(i+100)]
generated_text = gen_text(generative_model, start_text)

In [63]:
generated_text

"  Durkheim, they, the peasant, a little do they had nothing to gain, and even the linen's own which subjects behaved either selfishled in drawing any absolute monarchy as a counterpois become more voluming the Devil, both bence the power of government, as they forgeour theoletical holding society scared of labour, so the landlord, the solid may considered useful  This property of altruism and staring and appropriating out we shall her cast, buth their philosophizing of this factions As Saci the language of colonialism In fact, we see very specifically the bourgeoisie itself, to the Deich that of the commodity considers to draws on comprehend its political power for the different kinds of commodities that the British position of the protective other in the relation of  commodity to commodity In fact, the struggle of the proletariat Racism, too, people spin and hoperty, the modern viewer reasoning information helps must focus a member or or  any ways in production More interested indivi

In [1]:
from that_text_gen import model as generate_model


In [2]:
model = generate_model('marx')

In [3]:
model.load_model()

In [4]:
test = model.generate_text(num = -5)


1


In [22]:
test[100:]

'essary to one that things could not go on long in this regard.'